In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error

In [2]:
covid_file = 'Data/COVID-19_Data_by_ZIP_Code.csv'
covid_data = pd.read_csv(covid_file)

In [3]:
crime_file = 'Data/KCPD_Crime_Data_2020.csv'
crime_data = pd.read_csv(crime_file)

In [4]:
pop_file = 'Data/Population_by_Zipcode'
pop_data = pd.read_csv(pop_file)

In [5]:
crime_df = crime_data[['Report_No', "Zip Code"]]

In [6]:
crime_df = crime_df.rename(columns={'Zip Code': 'Zipcode'})

In [7]:
crime_df = crime_df.loc[crime_df['Zipcode'] > 64100]
crime_df = crime_df.loc[crime_df['Zipcode'] < 64168]
crime_df = crime_df.groupby('Zipcode').count()
crime_df

,Report_No
Zipcode,
64101.0,85
64102.0,45
64103.0,5
64104.0,4
64105.0,805
...,...
64161.0,208
64163.0,21
64164.0,21


In [8]:
pop_data = pop_data.loc[pop_data['city'] == "Kansas City"]
pop_df = pop_data.loc[pop_data['state'] == 'MO']
pop_df = pop_df.set_index('Zipcode')
pop_df

,Population,city,state,latitude,longitude
Zipcode,,,,,
64167,446.0,Kansas City,MO,39.317300,-94.48655
64154,10895.0,Kansas City,MO,39.261679,-94.63353
64163,748.0,Kansas City,MO,39.318350,-94.67635
64166,285.0,Kansas City,MO,39.322849,-94.50755
64145,5249.0,Kansas City,MO,38.896407,-94.59666
64151,26777.0,Kansas City,MO,39.215652,-94.63021
64102,0.0,Kansas City,MO,39.093854,-94.60411
64109,9647.0,Kansas City,MO,39.065954,-94.56643
64125,1389.0,Kansas City,MO,39.104887,-94.49541


In [9]:
covid_df = covid_data[['ZipCode', 'COVID19 Cases']]
covid_df = covid_df.rename(columns={'ZipCode': 'Zipcode'})
covid_df = covid_df.set_index('Zipcode')
covid_df

,COVID19 Cases
Zipcode,
64101,SUPP*
64102,SUPP*
64105,44
64106,119
64108,109
64109,80
64110,87
64111,123
64112,76


In [10]:
engine = create_engine('sqlite:///covid_db.sqlite', echo=False)

In [11]:
covid_df.to_sql(name='covid_stats', con=engine, if_exists='append', index=True)

In [12]:
pop_df.to_sql(name='pop_stats', con=engine, if_exists='append', index=True)

In [13]:
crime_df.to_sql(name='crime_stats', con=engine, if_exists='append', index=True)

In [14]:
engine.table_names()

['covid_stats', 'crime_stats', 'pop_stats']